In [1]:
import os
import json
from openai import OpenAI

In [2]:
# 모델별 클라이언트 설정
MODEL_CONFIGS = {
    "gemini": {
        "id": "gemini-2.5-pro",
        "api_key": os.environ.get("GEMINI_API_KEY"),  # Gemini API key
        "base_url": os.environ.get("GEMINI_BASE_URL")  # Gemini base URL
    },
    "gpt": {
        "id": "gpt-5",
        "api_key": os.environ.get("OPENAI_API_KEY", os.environ.get("OPENAI_API_KEY")),
        "base_url": os.environ.get("OPENAI_BASE_URL", os.environ.get("OPENAI_BASE_URL"))
    },
    "claude": {
        "id": "claude-sonnet-4-20250514",
        "api_key": os.environ.get("CLAUDE_API_KEY", os.environ.get("OPENAI_API_KEY")),
        "base_url": os.environ.get("CLAUDE_BASE_URL", os.environ.get("OPENAI_BASE_URL"))
    },
}

def get_client(model_name):
    """모델에 맞는 클라이언트 반환"""
    if model_name not in MODEL_CONFIGS:
        return None
    
    config = MODEL_CONFIGS[model_name]
    return OpenAI(
        api_key=config["api_key"],
        base_url=config["base_url"]
    )

def call_different_model(model_name, query, system_prompt="You are a helpful assistant."):
    """다른 모델을 호출하는 함수"""
    if model_name not in MODEL_CONFIGS:
        return f"모델 '{model_name}'을 찾을 수 없습니다. 사용 가능한 모델: {list(MODEL_CONFIGS.keys())}"
    
    config = MODEL_CONFIGS[model_name]
    client = get_client(model_name)
    
    if not client:
        return f"모델 '{model_name}'의 클라이언트를 생성할 수 없습니다."
    
    try:
        temp_messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ]
        
        response = client.chat.completions.create(
            model=config["id"],
            messages=temp_messages
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"모델 '{model_name}' 호출 중 오류: {str(e)}"

# 파이프라인용 모델 설정도 업데이트
PIPELINE_MODELS = {
    "flash": {
        "model_name": "gemini",
        "role": "초기 응답 생성기",
        "system_prompt": "You are a fast response generator. Provide comprehensive but raw information about the user's query. Include all relevant details. Respond in Korean."
    },
    "sonnet": {
        "model_name": "claude", 
        "role": "Markdown 정리자",
        "system_prompt": "You are a markdown formatter. Take the provided content and organize it into well-structured markdown format with proper headers, lists, and formatting. Maintain all information but improve readability. Respond in Korean."
    },
    "gpt": {
        "model_name": "gpt",  # gemini-2.5-pro 사용하려면 MODEL_CONFIGS에 추가 필요
        "role": "최종 조직자",
        "system_prompt": "You are a content organizer. Take the markdown content and reorganize it logically, add insights, remove redundancy, and present it in the most helpful way for the user. Respond in Korean."
    }
}

def run_pipeline_step(model_key, content, user_query=None):
    """파이프라인의 각 단계 실행"""
    if model_key not in PIPELINE_MODELS:
        return f"모델 '{model_key}'를 찾을 수 없습니다."
    
    pipeline_config = PIPELINE_MODELS[model_key]
    model_name = pipeline_config["model_name"]
    
    # 해당 모델의 클라이언트 가져오기
    client = get_client(model_name)
    if not client:
        return f"모델 '{model_name}'의 클라이언트를 생성할 수 없습니다."
    
    # 모델 ID 가져오기
    model_id = MODEL_CONFIGS[model_name]["id"]
    
    # 단계별 메시지 구성
    if model_key == "flash":
        messages = [
            {"role": "system", "content": pipeline_config["system_prompt"]},
            {"role": "user", "content": user_query}
        ]
    elif model_key == "sonnet":
        messages = [
            {"role": "system", "content": pipeline_config["system_prompt"]},
            {"role": "user", "content": f"다음 내용을 잘 정리된 마크다운 형태로 변환해주세요:\n\n{content}"}
        ]
    else:  # pro
        messages = [
            {"role": "system", "content": pipeline_config["system_prompt"]},
            {"role": "user", "content": f"다음 마크다운 내용을 더 체계적으로 재구성하고 인사이트를 추가해주세요. 사실관계가 포함된 내용의 경우 정확한 내용인지 팩트 체크를 진행해주세요:\n\n{content}"}
        ]
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"모델 '{model_key}' 호출 중 오류: {str(e)}"

# 기본 클라이언트 (오케스트레이터용)
default_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL")
)

In [3]:
def execute_pipeline(user_query):
    """전체 파이프라인 실행"""
    print("🔄 파이프라인 실행 중...")

    # 1단계: Pro - 초기 응답 생성
    print("1️⃣ Pro가 초기 응답을 생성 중...")
    flash_result = run_pipeline_step("flash", None, user_query)
    print(f"Flash 완료: {len(flash_result)} 문자")
    
    # 2단계: Sonnet - Markdown 정리
    print("2️⃣ Sonnet이 Markdown으로 정리 중...")
    sonnet_result = run_pipeline_step("sonnet", flash_result)
    print(f"Sonnet 완료: {len(sonnet_result)} 문자")
    
    # 3단계: Pro - 최종 조직화
    print("3️⃣ GPT가 최종 조직화 중...")
    pro_result = run_pipeline_step("gpt", sonnet_result)
    print(f"Pro 완료: {len(pro_result)} 문자")
    
    return {
        "pro": flash_result,
        "sonnet": sonnet_result, 
        "gpt": pro_result
    }

# tools 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_pipeline",
            "description": "Flash → Sonnet → Pro 파이프라인을 실행하여 질문에 대한 체계적인 답변을 생성합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_query": {
                        "type": "string",
                        "description": "사용자의 질문이나 요청"
                    }
                },
                "required": ["user_query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "run_pipeline_step", 
            "description": "파이프라인의 특정 단계만 실행합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "model_key": {
                        "type": "string",
                        "enum": ["flash", "sonnet", "pro"],
                        "description": "실행할 모델 단계"
                    },
                    "content": {
                        "type": "string",
                        "description": "처리할 내용"
                    },
                    "user_query": {
                        "type": "string", 
                        "description": "원본 사용자 질문 (flash 단계에서만 필요)"
                    }
                },
                "required": ["model_key", "content"]
            }
        }
    }
]

FUNCTION_MAP = {
    "execute_pipeline": execute_pipeline,
    "run_pipeline_step": run_pipeline_step
}

messages = [
    {"role": "system", "content": "당신은 Flash → Sonnet → Pro 파이프라인을 관리하는 오케스트레이터입니다. 사용자의 복잡한 질문이나 요청에 대해 체계적인 답변이 필요할 때 파이프라인을 실행하세요."}
]

print("🤖 3단계 파이프라인 에이전트 시작")
print("Flash(초기응답) → Sonnet(마크다운정리) → Pro(최종조직화)")
print("종료: exit")

while True:
    message = input("\nEnter your message: ").strip()
    
    if message == 'exit':
        break
    if not message:
        continue

    messages.append({"role": "user", "content": message})

    try:
        response = default_client.chat.completions.create(
            model="gpt-5-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        tool_calls = getattr(response_message, "tool_calls", None)
        
        if tool_calls:
            messages.append(response_message.model_dump(exclude_none=True))
            
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments or "{}")
                
                if function_name == "execute_pipeline":
                    user_query = function_args.get("user_query", message)
                    pipeline_results = execute_pipeline(user_query)
                    
                    print("\n" + "="*50)
                    print("📋 최종 결과:")
                    print("="*50)
                    print(pipeline_results["pro"])
                    
                    # 중간 결과도 저장 (원하면 출력 가능)
                    function_result = f"파이프라인 실행 완료. 최종 결과: {pipeline_results['pro']}"
                    
                elif function_name == "run_pipeline_step":
                    model_key = function_args.get("model_key") 
                    content = function_args.get("content")
                    user_query = function_args.get("user_query")
                    
                    step_result = run_pipeline_step(model_key, content, user_query)
                    print(f"\n{model_key.upper()} 결과:")
                    print("-" * 30)
                    print(step_result)
                    
                    function_result = step_result
                else:
                    function_result = f"알 수 없는 함수: {function_name}"
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_result
                })
            
            # 최종 오케스트레이터 응답
            final_response = default_client.chat.completions.create(
                model="gpt-5-mini",
                messages=messages
            )
            
            assistant_message = final_response.choices[0].message.content
            print(f"\n🎭 Orchestrator: {assistant_message}")
            messages.append({"role": "assistant", "content": assistant_message})
            
        else:
            print(f"Assistant: {response_message.content}")
            messages.append({"role": "assistant", "content": response_message.content})
            
    except Exception as e:
        print(f"API 오류: {e}")
        messages.pop()

🤖 3단계 파이프라인 에이전트 시작
Flash(초기응답) → Sonnet(마크다운정리) → Pro(최종조직화)
종료: exit
🔄 파이프라인 실행 중...
1️⃣ Pro가 초기 응답을 생성 중...
Flash 완료: 2886 문자
2️⃣ Sonnet이 Markdown으로 정리 중...
Sonnet 완료: 38 문자
3️⃣ GPT가 최종 조직화 중...
Pro 완료: 2814 문자

📋 최종 결과:
## 켈로그(Kellogg's) vs 포스트(Post) 대립 관계 및 현황

켈로그와 포스트는 100년 넘게 이어진 시리얼 시장의 양대 산맥이자 숙명의 라이벌입니다. 두 기업의 대립 관계와 현재 우위는 글로벌 시장과 한국 시장에서 다소 다른 양상을 보입니다.

### 1. 대립의 역사: 한 병원에서 시작된 전쟁

*   **시작:** 19세기 말, 의사 존 하비 켈로그(John Harvey Kellogg)는 요양원을 운영하며 환자들을 위한 건강 시리얼을 개발했습니다. 이때 그의 요양원에 환자로 입원했던 C.W. 포스트(Charles William Post)가 시리얼의 상업적 가능성을 보고 퇴원 후 먼저 '포스트'를 창업합니다.
*   **켈로그의 반격:** 이에 자극받은 존 하비 켈로그의 동생 윌 키스 켈로그(Will Keith Kellogg)가 형의 아이디어를 바탕으로 '켈로그'를 설립하면서 두 회사의 본격적인 경쟁이 시작됩니다. 즉, 한 동네(미국 배틀크리크)의 한 요양원에서 모든 것이 시작된 셈입니다.
*   **경쟁 양상:**
    *   **초기:** 서로의 제품을 모방하고 개선하는 '카피캣' 전략이 주를 이뤘습니다. (예: 켈로그 콘플레이크 vs 포스트 토스티스)
    *   **중기 (20세기):** TV 광고 시대가 열리면서 '토니 더 타이거'(켈로그)와 같은 상징적인 캐릭터를 앞세운 마케팅 전쟁이 치열했습니다. 아이들을 타겟으로 한 달콤한 시리얼 경쟁이 극에 달했습니다.
    *   **후기~현재:** 건강에 대한 관심이 높아지면서 그래놀라, 통

In [11]:
message = input("Enter your message: ").strip()
messages.append({"role": "user", "content": message})
response = default_client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  stream=True
)

for chunk in response:
  print(chunk.choices[0].delta.content, end='', flush=True)

오레오오즈가 인기를 끌었던 이유는 여러 가지가 있습니다:

1. **독특한 맛과 구성**: 오레오 오즈 시리얼은 오레오 쿠키의 달콤하고 크리미한 맛을 재현하여 친숙하고 매력적인 맛을 제공합니다. 크리스피한 식감과 초코맛 링에 부드러운 마시멜로 토핑은 소비자들에게 색다르고 즐거운 경험을 줍니다.

2. **브랜드 파워**: 오레오는 전 세계적으로 유명한 쿠키 브랜드로, 이미 많은 팬층을 보유하고 있습니다. 이 브랜드 인지도는 시리얼 형태의 변주 제품인 오레오 오즈가 빠르게 인지도를 갖추고 소비자에게 다가가는데 도움을 주었습니다.

3. **한정 생산과 가성비**: 한동안 오레오 오즈는 미국에서는 판매되지 않으며 한국에서만 생산되어, 해외 직구템으로 주목받으며 희소가치를 가졌습니다. 이로 인해 한국에서는 더욱 인기가 상승했습니다.

4. **적절한 마케팅**: 동서식품은 한국 소비자의 취향과 트렌드를 잘 분석하여 제품을 출시하고 홍보하였습니다. 특히 온라인에서 입소문을 타면서 더욱 인기도가 상승했습니다.

5. **시각적 매력**: 제품의 비주얼 자체도 소비자, 특히 젊은 층의 취향에 맞았고, SNS나 온라인 리뷰를 통한 자연스러운 확산에 기여했습니다.

이들 요소가 결합되어 오레오 오즈는 많은 사랑을 받게 되었고, 이러한 인기가 꾸준히 유지되고 있습니다.None